In [0]:
import cv2 as cv
from google.colab.patches import cv2_imshow
from PIL import Image
from scipy import ndimage
import copy

In [0]:
car_file = 'car.jpg'
city_file = 'citymap.png'

In [0]:
class car(object):

  #x and y are center points of the car
  def __init__(self, x, y, angle):
    self.x = x
    self.y = y
    self.length, self.width = int(20), int(10)
    self.angle = angle
  
  def move(self, velocity_x, velocity_y, rotation):
    self.x = self.x + velocity_x
    self.y = self.y + velocity_y
    self.angle = self.angle + rotation
     

In [0]:
class city(object):

  def __init__(self, city_file):
    self.city_file = city_file
    self.city_img = cv.imread(self.city_file)
    self.width, self.length, _ = self.city_img.shape



In [0]:
class env(object):

  def __init__(self, car, city, car_img):
    
    self.car = car
    self.city = city
    self.car_img = car_img
    self.car_img = cv.resize(self.car_img,(self.car.length, self.car.width))

  #car_x and car_y are center points of the car
  def show_image(self):   
    newcity = copy.deepcopy(self.city)
    
    car_rotated = ndimage.rotate(self.car_img, self.car.angle)
    car_wid, car_len, _ = car_rotated.shape

    pos_x = self.car.x - (car_len//2)
    pos_y = newcity.width - (self.car.y - (car_wid//2))
    print(pos_x, pos_y, car_wid, newcity.length, newcity.width)
    
    if (pos_x < 0):
      pos_x = 0
    elif(pos_x >newcity.length):
      pos_x = newcity.length - car_len

    if(pos_y > newcity.width):
      pos_y = newcity.width - car_wid
    elif(pos_y < 0):
      pos_y = 0
    
    print(pos_x, pos_y, car_wid, car_len)

    car_rotated = cv.addWeighted(newcity.city_img[pos_y:pos_y+car_wid, pos_x:pos_x+car_len],0.5,car_rotated,1,0)
    newcity.city_img[pos_y:pos_y+car_wid, pos_x:pos_x+car_len] = car_rotated
    return newcity

    

In [6]:
car_img = cv.imread(car_file)
car1 = car(200,200,30)
city1 = city(city_file)
a = env(car1, city1, car_img)
newcity = a.show_image()
cv2_imshow(newcity.city_img)

189 469 19 1429 660
189 469 19 22


In [7]:
car1.move(100,100,30)
newcity1 = a.show_image()
cv2_imshow(newcity1.city_img)

291 371 22 1429 660
291 371 22 19


In [8]:
car = cv.imread(car_file)
newcar = cv.resize(car,(int(20), int(10)))
img_rotated = ndimage.rotate(newcar, 60)
cv2_imshow(img_rotated)
cv2_imshow(newcar)

print(img_rotated.shape)

(22, 19, 3)
